In [1]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v2.py
import os, re, glob
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

import numpy as np
import pandas as pd
import tensorflowjs as tfjs

from keras import layers, Model, callbacks
from keras.optimizers import Adam
from keras.applications.mobilenetv2 import MobileNetV2
from sklearn.model_selection import train_test_split
from ohia.metrics import top_1_accuracy, top_3_accuracy, top_5_accuracy
from ohia.encoders import FastLabelEncoder
from ohia.utils import PlantNetGenerator

Using TensorFlow backend.


In [7]:
SEED = 0
ALPHA = 1.4 # [0.35, 0.50, 0.75, 1.0, 1.3, 1.4]
BATCH_SIZE = 64
MODEL_NAME = f'mobilenetv2-{ALPHA:0.2f}'
FILE_PATH = '/home/matt/repos/ohia.ai/data'
IMAGE_DIR = 'resized_images'

In [3]:
# get list of images and labels
file_list = glob.glob(f'{FILE_PATH}/{IMAGE_DIR}/**/*.jpg', recursive=True)
full_label_list = [re.split('/', f)[-2] for f in file_list]
full_label_ids = FastLabelEncoder().fit_transform(full_label_list)

In [4]:
# split data
train_files, valid_files, train_ids, valid_ids \
    = train_test_split(file_list, full_label_ids, test_size=0.1, random_state=SEED)

In [5]:
# create generators
n_classes = len(np.unique(full_label_list))
train_generator = PlantNetGenerator(train_files, train_ids, n_classes, BATCH_SIZE, augment=True)
valid_generator = PlantNetGenerator(valid_files, valid_ids, n_classes, BATCH_SIZE, augment=False)

In [8]:
# load pretrained ImageNet model
base_model = MobileNetV2(
    input_shape=(224,224,3),
    alpha=ALPHA,
    weights='imagenet',
    include_top=False
)

# set freeze all layers
for layer in base_model.layers:
    layer.trainable = False
    
# map ImageNet features to plants
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(2048, activation='relu')(x)
outputs = layers.Dense(n_classes, activation='softmax')(x)
    
# compile the model
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=[top_1_accuracy, top_3_accuracy, top_5_accuracy]
)

# define callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_top_3_accuracy',
        patience=10,
        verbose=1,
        mode='max',
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_top_3_accuracy',
        factor=0.5,
        patience=3,
        verbose=1,
        mode='max',
    ),
    callbacks.ModelCheckpoint(
        monitor='val_top_3_accuracy',
        filepath=f'{FILE_PATH}/weights/{MODEL_NAME}' + '_{epoch:02d}.hdf5',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
    ),
]

# train model
model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    use_multiprocessing=True,
    workers=10,
    epochs=100,    
)

Epoch 1/100
165/165 [==============================] - 50s 300ms/step - loss: 1.6453 - top_1_accuracy: 0.5441 - top_3_accuracy: 0.7402 - top_5_accuracy: 0.8210 - val_loss: 1.4767 - val_top_1_accuracy: 0.5365 - val_top_3_accuracy: 0.7925 - val_top_5_accuracy: 0.8845
Epoch 2/100
165/165 [==============================] - 45s 273ms/step - loss: 1.0016 - top_1_accuracy: 0.7116 - top_3_accuracy: 0.8813 - top_5_accuracy: 0.9333 - val_loss: 1.4496 - val_top_1_accuracy: 0.5738 - val_top_3_accuracy: 0.7934 - val_top_5_accuracy: 0.8837
Epoch 3/100
165/165 [==============================] - 46s 276ms/step - loss: 0.8497 - top_1_accuracy: 0.7453 - top_3_accuracy: 0.9056 - top_5_accuracy: 0.9523 - val_loss: 1.3499 - val_top_1_accuracy: 0.5911 - val_top_3_accuracy: 0.8203 - val_top_5_accuracy: 0.9062
Epoch 4/100
165/165 [==============================] - 46s 280ms/step - loss: 0.7425 - top_1_accuracy: 0.7777 - top_3_accuracy: 0.9240 - top_5_accuracy: 0.9641 - val_loss: 1.3564 - val_top_1_accuracy: 0

In [9]:
# load best model
best_weights = glob.glob(f'{FILE_PATH}/weights/{MODEL_NAME}**')
best_weights = np.sort(best_weights)[-1]
model.load_weights(best_weights)

In [10]:
# save model
tfjs.converters.save_keras_model(model, f'{FILE_PATH}/models/{MODEL_NAME}')

In [ ]:
# alpha: 1.40,  val_top_1_accuracy: 0.6068,  val_top_3_accuracy: 0.8490,  val_top_5_accuracy: 0.9149
# alpha: 1.00,  val_top_1_accuracy: 0.5521,  val_top_3_accuracy: 0.7786,  val_top_5_accuracy: 0.8698